---
format:
  html:
    embed-resources: true
    standalone: true 
    toc: true
    toc-location: right
    smooth-scroll: true
    linestretch: 1.7
execute: 
  echo: false
  cache: true
theme: 
  - united
jupyter: python3
include-in-header:
  text: |
    <script src="https://cdn.jsdelivr.net/npm/ol@v10.1.0/dist/ol.js"></script>
    <script src="https://cdn.jsdelivr.net/npm/geotiff@2.1.3/dist-browser/geotiff.min.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/openlayers/10.1.0/renderer/canvas/TileLayer.min.js" integrity="sha512-ZFGaxTCNEAK36iyWVISk3ld5vrfhs6x1WFxQyZs9i79l2RHbaxuktxWop4MWwYoBJvM9cb/ReU3iBJIgwxrY0w==" crossorigin="anonymous" referrerpolicy="no-referrer"></script>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/ol@v10.1.0/ol.css">
---

In [1]:
location = ""
version = ""
geometries = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              -81.11207031421274,
              42.68616218382104
            ],
            [
              -81.11207031421274,
              42.67880724579865
            ],
            [
              -81.10653179917108,
              42.67880724579865
            ],
            [
              -81.10653179917108,
              42.68616218382104
            ],
            [
              -81.11207031421274,
              42.68616218382104
            ]
          ]
        ],
        "type": "Polygon"
      }
    },
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              -81.10024380542592,
              42.672904126895986
            ],
            [
              -81.10024380542592,
              42.664430930623496
            ],
            [
              -81.09789142538074,
              42.664430930623496
            ],
            [
              -81.09789142538074,
              42.672904126895986
            ],
            [
              -81.10024380542592,
              42.672904126895986
            ]
          ]
        ],
        "type": "Polygon"
      }
    },
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              -81.12204202201008,
              42.669649213361254
            ],
            [
              -81.12204202201008,
              42.66649636597094
            ],
            [
              -81.11858789434974,
              42.66649636597094
            ],
            [
              -81.11858789434974,
              42.669649213361254
            ],
            [
              -81.12204202201008,
              42.669649213361254
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

In [3]:
import geopandas as gpd
import plotly.express as px
import plotly.io as pio
import pandas as pd
import plotly.graph_objects as go
from geopy.geocoders import Nominatim
import asyncio
import rasterstats
import json
import environmental_risk_metrics

pio.renderers.default = "notebook"


In [4]:
gdf = gpd.GeoDataFrame.from_features(geometries["features"], crs="EPSG:4326")
bounds_list = gdf.total_bounds.tolist()
bounds = [[bounds_list[0], bounds_list[1]], [bounds_list[2], bounds_list[3]]]
center = [(bounds[0][0] + bounds[1][0]) / 2, (bounds[0][1] + bounds[1][1]) / 2]

In [6]:
locator = Nominatim(user_agent="RegenRate.com")
location = locator.reverse(f"{center[1]}, {center[0]}", language="en")
location = str(location.raw["display_name"])
location


"Lucy's Trail, Central Elgin, Elgin County, Ontario, N5P 3S8, Canada"

In [7]:
%load_ext autoreload
%autoreload 2

In [8]:
# import plotly.express as px
# fig = go.Figure()
# # change figure height
# fig.update_layout(height=10)
# fig

In [9]:
from environmental_risk_metrics import (
    Sentinel2,
    EsaLandCover,
    EsriLandCover,
    OpenLandMapLandCover,
    SoilOrganicCarbon,
    SoilTypes,
    EndangeredSpecies,
    RamsarProtectedAreas,
    GlobalWitness,
    SoilOrganicCarbonPotential,
)


start_date = "2015-06-01"
end_date = "2024-08-31"

polygon_crs = "EPSG:4326"


for metric in [
    # Sentinel2,
    EsaLandCover,
    EsriLandCover,
    OpenLandMapLandCover,
    SoilOrganicCarbon,
    SoilTypes,
    EndangeredSpecies,
    RamsarProtectedAreas,
    GlobalWitness,
    SoilOrganicCarbonPotential,
]:
    print(metric.__name__)
    metric_instance = metric(
        gdf=gdf,
    )

    metric_instance.get_data(
        start_date=start_date,
        end_date=end_date,
    )


EsaLandCover


100%|██████████| 6/6 [00:00<00:00, 104.44it/s]


EsriLandCover


100%|██████████| 7/7 [00:00<00:00, 40.17it/s]


OpenLandMapLandCover


INFO:GDAL signalled an error: err_no=1, msg="Line 0: </head> doesn't have matching <head>."


RasterioIOError: Line 0: </head> doesn't have matching <head>.

In [7]:
from datetime import datetime
from IPython.display import HTML

def format_date_with_suffix(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    
    formatted_date = date.strftime(f"%B {day}{suffix}, %Y")
    return formatted_date

current_date = datetime.now()
current_date = format_date_with_suffix(current_date)

HTML(f"""<div style="max-width: 900px; margin: 50px auto; padding: 20px; background-color: #fff; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1); border-radius: 8px; font-family: Arial, sans-serif;">
  <h1 style="text-align: center; font-size: 2.5rem; margin-bottom: 20px; color: #2c3e50;">Risk Assessment Report</h1>
  <div style="text-align: center; font-size: 1.2rem; margin-bottom: 30px; color: #34495e;">
    <div>{current_date}</div>
    <div>{location}</div>
  </div>
</div>""")

In [11]:
import leafmap
m = leafmap.Map(
    center=(center[1], center[0]),
    zoom=12,
    draw_control=False,
    measure_control=False,
    fullscreen_control=False,
    attribution_control=False,
    search_control=False,
    layers_control=False,
    scale_control=False,
    toolbar_control=False,
    basemap="Esri.WorldImagery"
)
m.add_geojson(gdf, layer_name="Your Parcels", zoom_to_layer=True)
m

Map(center=[42.67529655722227, -81.10996672369541], controls=(ZoomControl(options=['position', 'zoom_in_text',…

This report provides an automated assessment of the risk associated with land use changes in the specified parcel, based on ISCC Principle I criteria.

ISCC Principle I mandates the protection of areas with high biodiversity value and high carbon stock, ensuring that biofuel production does not compromise these critical environments. The analysis covers the conversion of land, proximity to protected areas, impact on carbon stocks, and potential threats to biodiversity.

The risk level is determined through a combination of satellite imagery, geospatial overlays, and biodiversity impact assessments, offering a comprehensive evaluation of compliance with ISCC sustainability requirements.

{{< pagebreak >}}


## Land Use Change
> The following section provides an overview of land use changes in the area of interest. The analysis includes a per field analysis of land cover changes over time and an overall assessment of land use change in the region.

### Overall Land Use Change
> The chart below shows the land use change for each field over time. Click on a field to get a more detailed view. Red indicates a high risk of land use change, while green represents low risk. High risk areas are those with land use change from a rich and biodiverse ecosystem to a less diverse one.


In [ ]:
#| fig-align: center
from colors import land_cover_color_mapping
risk_assessment_fig = px.sunburst(
    risk_assessments_df,
    path=["Field", "Year", "Land Cover", "Risk Assessment"],
    values="Percentage",
    color="Land Cover",
    color_discrete_map=color_mapping,
    custom_data=["Risk Reasoning"],
)

risk_assessment_fig.update_layout(autosize=False, width=600, height=600)

risk_assessment_fig.update_traces(
    marker_colors=[color_mapping.get(x, "#D3D3D3") for x in risk_assessment_fig.data[-1].labels],
    hovertemplate='<b>%{label}</b><br>Percentage: %{value}<br>Risk Reasoning: %{customdata[0]}<extra></extra>'
)
risk_assessment_fig.show()

### Land Use Maps
> Click on the layer button in the top right corner to switch between different years of the land cover maps. The legend provides information on the land cover classes and their corresponding colors.

::: column-screen

In [ ]:
import leafmap
import json

lc_glad = {
  "2000": "https://s3.openlandmap.org/arco/lc_glad.glcluc_c_30m_s_20000101_20001231_go_epsg.4326_v20230901.tif",
  "2005": "https://s3.openlandmap.org/arco/lc_glad.glcluc_c_30m_s_20050101_20051231_go_epsg.4326_v20230901.tif",
  "2010": "https://s3.openlandmap.org/arco/lc_glad.glcluc_c_30m_s_20100101_20101231_go_epsg.4326_v20230901.tif",
  "2015": "https://s3.openlandmap.org/arco/lc_glad.glcluc_c_30m_s_20150101_20151231_go_epsg.4326_v20230901.tif",
  "2020": "https://s3.openlandmap.org/arco/lc_glad.glcluc_c_30m_s_20200101_20201231_go_epsg.4326_v20230901.tif",
}


m = leafmap.Map(
    center=(center[1], center[0]),
    zoom=12,
    draw_control=False,
    measure_control=False,
    fullscreen_control=False,
    attribution_control=False,
    search_control=False,
    layers_control=True,
    scale_control=False,
    toolbar_control=False,
)

legend = [
 {'value': '1',
  'color': '#fafac3',
  'label': 'Terra Firma, true desert - 7% short veg. cover (1)'},
 {'value': '2',
  'color': '#f7f7bb',
  'label': 'Terra Firma, semi-arid - 11% short veg. cover (2)'},
 {'value': '3',
  'color': '#f4f4b3',
  'label': 'Terra Firma, semi-arid - 15% short veg. cover (3)'},
 {'value': '4',
  'color': '#f1f1ab',
  'label': 'Terra Firma, semi-arid - 19% short veg. cover (4)'},
 {'value': '5',
  'color': '#ededa2',
  'label': 'Terra Firma, semi-arid - 23% short veg. cover (5)'},
 {'value': '6',
  'color': '#eaea9a',
  'label': 'Terra Firma, semi-arid - 27% short veg. cover (6)'},
 {'value': '7',
  'color': '#e7e792',
  'label': 'Terra Firma, semi-arid - 31% short veg. cover (7)'},
 {'value': '8',
  'color': '#e4e48a',
  'label': 'Terra Firma, semi-arid - 35% short veg. cover (8)'},
 {'value': '9',
  'color': '#e0e081',
  'label': 'Terra Firma, semi-arid - 39% short veg. cover (9)'},
 {'value': '10',
  'color': '#dddd79',
  'label': 'Terra Firma, semi-arid - 43% short veg. cover (10)'},
 {'value': '11',
  'color': '#dada71',
  'label': 'Terra Firma, semi-arid - 47% short veg. cover (11)'},
 {'value': '12',
  'color': '#d7d769',
  'label': 'Terra Firma, semi-arid - 51% short veg. cover (12)'},
 {'value': '13',
  'color': '#d3d360',
  'label': 'Terra Firma, semi-arid - 55% short veg. cover (13)'},
 {'value': '14',
  'color': '#d0d058',
  'label': 'Terra Firma, semi-arid - 59% short veg. cover (14)'},
 {'value': '15',
  'color': '#cdcd50',
  'label': 'Terra Firma, semi-arid - 63% short veg. cover (15)'},
 {'value': '16',
  'color': '#caca48',
  'label': 'Terra Firma, semi-arid - 67% short veg. cover (16)'},
 {'value': '17',
  'color': '#c6c63f',
  'label': 'Terra Firma, semi-arid - 71% short veg. cover (17)'},
 {'value': '18',
  'color': '#c3c337',
  'label': 'Terra Firma, semi-arid - 75% short veg. cover (18)'},
 {'value': '19',
  'color': '#c0c02f',
  'label': 'Terra Firma, dense short vegetation - 79% short veg. cover (19)'},
 {'value': '20',
  'color': '#bdbd27',
  'label': 'Terra Firma, dense short vegetation - 83% short veg. cover (20)'},
 {'value': '21',
  'color': '#b9b91e',
  'label': 'Terra Firma, dense short vegetation - 87% short veg. cover (21)'},
 {'value': '22',
  'color': '#b6b616',
  'label': 'Terra Firma, dense short vegetation - 91% short veg. cover (22)'},
 {'value': '23',
  'color': '#b3b30e',
  'label': 'Terra Firma, dense short vegetation - 95% short veg. cover (23)'},
 {'value': '24',
  'color': '#b0b006',
  'label': 'Terra Firma, dense short vegetation - 100% short veg. cover (24)'},
 {'value': '25',
  'color': '#609c60',
  'label': 'Terra Firma, stable tree cover - 3m trees (25)'},
 {'value': '26',
  'color': '#5c985c',
  'label': 'Terra Firma, stable tree cover - 4m trees (26)'},
 {'value': '27',
  'color': '#589558',
  'label': 'Terra Firma, stable tree cover - 5m trees (27)'},
 {'value': '28',
  'color': '#549254',
  'label': 'Terra Firma, stable tree cover - 6m trees (28)'},
 {'value': '29',
  'color': '#508e50',
  'label': 'Terra Firma, stable tree cover - 7m trees (29)'},
 {'value': '30',
  'color': '#4c8b4c',
  'label': 'Terra Firma, stable tree cover - 8m trees (30)'},
 {'value': '31',
  'color': '#488848',
  'label': 'Terra Firma, stable tree cover - 9m trees (31)'},
 {'value': '32',
  'color': '#448544',
  'label': 'Terra Firma, stable tree cover - 10m trees (32)'},
 {'value': '33',
  'color': '#408140',
  'label': 'Terra Firma, stable tree cover - 11m trees (33)'},
 {'value': '34',
  'color': '#3c7e3c',
  'label': 'Terra Firma, stable tree cover - 12m trees (34)'},
 {'value': '35',
  'color': '#387b38',
  'label': 'Terra Firma, stable tree cover - 13m trees (35)'},
 {'value': '36',
  'color': '#347834',
  'label': 'Terra Firma, stable tree cover - 14m trees (36)'},
 {'value': '37',
  'color': '#317431',
  'label': 'Terra Firma, stable tree cover - 15m trees (37)'},
 {'value': '38',
  'color': '#2d712d',
  'label': 'Terra Firma, stable tree cover - 16m trees (38)'},
 {'value': '39',
  'color': '#296e29',
  'label': 'Terra Firma, stable tree cover - 17m trees (39)'},
 {'value': '40',
  'color': '#256b25',
  'label': 'Terra Firma, stable tree cover - 18m trees (40)'},
 {'value': '41',
  'color': '#216721',
  'label': 'Terra Firma, stable tree cover - 19m trees (41)'},
 {'value': '42',
  'color': '#1d641d',
  'label': 'Terra Firma, stable tree cover - 20m trees (42)'},
 {'value': '43',
  'color': '#196119',
  'label': 'Terra Firma, stable tree cover - 21m trees (43)'},
 {'value': '44',
  'color': '#155e15',
  'label': 'Terra Firma, stable tree cover - 22m trees (44)'},
 {'value': '45',
  'color': '#115a11',
  'label': 'Terra Firma, stable tree cover - 23m trees (45)'},
 {'value': '46',
  'color': '#0d570d',
  'label': 'Terra Firma, stable tree cover - 24m trees (46)'},
 {'value': '47',
  'color': '#095409',
  'label': 'Terra Firma, stable tree cover - 25m trees (47)'},
 {'value': '48',
  'color': '#065106',
  'label': 'Terra Firma, stable tree cover - >25m trees (48)'},
 {'value': '49',
  'color': '#643700',
  'label': 'Terra Firma, tree cover with prev. disturb. (2020 height) - 3m trees (49)'},
 {'value': '50',
  'color': '#643a00',
  'label': 'Terra Firma, tree cover with prev. disturb. (2020 height) - 4m trees (50)'},
 {'value': '51',
  'color': '#643d00',
  'label': 'Terra Firma, tree cover with prev. disturb. (2020 height) - 5m trees (51)'},
 {'value': '52',
  'color': '#644000',
  'label': 'Terra Firma, tree cover with prev. disturb. (2020 height) - 6m trees (52)'},
 {'value': '53',
  'color': '#644300',
  'label': 'Terra Firma, tree cover with prev. disturb. (2020 height) - 7m trees (53)'},
 {'value': '54',
  'color': '#644600',
  'label': 'Terra Firma, tree cover with prev. disturb. (2020 height) - 8m trees (54)'},
 {'value': '55',
  'color': '#644900',
  'label': 'Terra Firma, tree cover with prev. disturb. (2020 height) - 9m trees (55)'},
 {'value': '56',
  'color': '#654c00',
  'label': 'Terra Firma, tree cover with prev. disturb. (2020 height) - 10m trees (56)'},
 {'value': '57',
  'color': '#654f00',
  'label': 'Terra Firma, tree cover with prev. disturb. (2020 height) - 11m trees (57)'},
 {'value': '58',
  'color': '#655200',
  'label': 'Terra Firma, tree cover with prev. disturb. (2020 height) - 12m trees (58)'},
 {'value': '59',
  'color': '#655500',
  'label': 'Terra Firma, tree cover with prev. disturb. (2020 height) - 13m trees (59)'},
 {'value': '60',
  'color': '#655800',
  'label': 'Terra Firma, tree cover with prev. disturb. (2020 height) - 14m trees (60)'},
 {'value': '61',
  'color': '#655a00',
  'label': 'Terra Firma, tree cover with prev. disturb. (2020 height) - 15m trees (61)'},
 {'value': '62',
  'color': '#655d00',
  'label': 'Terra Firma, tree cover with prev. disturb. (2020 height) - 16m trees (62)'},
 {'value': '63',
  'color': '#656000',
  'label': 'Terra Firma, tree cover with prev. disturb. (2020 height) - 17m trees (63)'},
 {'value': '64',
  'color': '#656300',
  'label': 'Terra Firma, tree cover with prev. disturb. (2020 height) - 18m trees (64)'},
 {'value': '65',
  'color': '#666600',
  'label': 'Terra Firma, tree cover with prev. disturb. (2020 height) - 19m trees (65)'},
 {'value': '66',
  'color': '#666900',
  'label': 'Terra Firma, tree cover with prev. disturb. (2020 height) - 20m trees (66)'},
 {'value': '67',
  'color': '#666c00',
  'label': 'Terra Firma, tree cover with prev. disturb. (2020 height) - 21m trees (67)'},
 {'value': '68',
  'color': '#666f00',
  'label': 'Terra Firma, tree cover with prev. disturb. (2020 height) - 22m trees (68)'},
 {'value': '69',
  'color': '#667200',
  'label': 'Terra Firma, tree cover with prev. disturb. (2020 height) - 23m trees (69)'},
 {'value': '70',
  'color': '#667500',
  'label': 'Terra Firma, tree cover with prev. disturb. (2020 height) - 24m trees (70)'},
 {'value': '71',
  'color': '#667800',
  'label': 'Terra Firma, tree cover with prev. disturb. (2020 height) - 25m trees (71)'},
 {'value': '72',
  'color': '#667b00',
  'label': 'Terra Firma, tree cover with prev. disturb. (2020 height) - >25m trees (72)'},
 {'value': '73',
  'color': '#ff99ff',
  'label': 'Terra Firma, tree height gain (2020 height) - 3m trees (73)'},
 {'value': '74',
  'color': '#fc92fc',
  'label': 'Terra Firma, tree height gain (2020 height) - 4m trees (74)'},
 {'value': '75',
  'color': '#f98bf9',
  'label': 'Terra Firma, tree height gain (2020 height) - 5m trees (75)'},
 {'value': '76',
  'color': '#f685f6',
  'label': 'Terra Firma, tree height gain (2020 height) - 6m trees (76)'},
 {'value': '77',
  'color': '#f37ef3',
  'label': 'Terra Firma, tree height gain (2020 height) - 7m trees (77)'},
 {'value': '78',
  'color': '#f077f0',
  'label': 'Terra Firma, tree height gain (2020 height) - 8m trees (78)'},
 {'value': '79',
  'color': '#ed71ed',
  'label': 'Terra Firma, tree height gain (2020 height) - 9m trees (79)'},
 {'value': '80',
  'color': '#ea6aea',
  'label': 'Terra Firma, tree height gain (2020 height) - 10m trees (80)'},
 {'value': '81',
  'color': '#e763e7',
  'label': 'Terra Firma, tree height gain (2020 height) - 11m trees (81)'},
 {'value': '82',
  'color': '#e45de4',
  'label': 'Terra Firma, tree height gain (2020 height) - 12m trees (82)'},
 {'value': '83',
  'color': '#e156e1',
  'label': 'Terra Firma, tree height gain (2020 height) - 13m trees (83)'},
 {'value': '84',
  'color': '#de4fde',
  'label': 'Terra Firma, tree height gain (2020 height) - 14m trees (84)'},
 {'value': '85',
  'color': '#db49db',
  'label': 'Terra Firma, tree height gain (2020 height) - 15m trees (85)'},
 {'value': '86',
  'color': '#d842d8',
  'label': 'Terra Firma, tree height gain (2020 height) - 16m trees (86)'},
 {'value': '87',
  'color': '#d53bd5',
  'label': 'Terra Firma, tree height gain (2020 height) - 17m trees (87)'},
 {'value': '88',
  'color': '#d235d2',
  'label': 'Terra Firma, tree height gain (2020 height) - 18m trees (88)'},
 {'value': '89',
  'color': '#cf2ecf',
  'label': 'Terra Firma, tree height gain (2020 height) - 19m trees (89)'},
 {'value': '90',
  'color': '#cc27cc',
  'label': 'Terra Firma, tree height gain (2020 height) - 20m trees (90)'},
 {'value': '91',
  'color': '#c921c9',
  'label': 'Terra Firma, tree height gain (2020 height) - 21m trees (91)'},
 {'value': '92',
  'color': '#c61ac6',
  'label': 'Terra Firma, tree height gain (2020 height) - 22m trees (92)'},
 {'value': '93',
  'color': '#c313c3',
  'label': 'Terra Firma, tree height gain (2020 height) - 23m trees (93)'},
 {'value': '94',
  'color': '#c00dc0',
  'label': 'Terra Firma, tree height gain (2020 height) - 24m trees (94)'},
 {'value': '95',
  'color': '#bd06bd',
  'label': 'Terra Firma, tree height gain (2020 height) - 25m trees (95)'},
 {'value': '96',
  'color': '#bb00bb',
  'label': 'Terra Firma, tree height gain (2020 height) - >25m trees (96)'},
 {'value': '100',
  'color': '#bfc0c0',
  'label': 'Wetland, salt pan - 3% short veg. cover (100)'},
 {'value': '101',
  'color': '#b7bdc2',
  'label': 'Wetland, salt pan - 7% short veg. cover (101)'},
 {'value': '102',
  'color': '#afbbc4',
  'label': 'Wetland, sparse vegetation - 11% short veg. cover (102)'},
 {'value': '103',
  'color': '#a8b8c6',
  'label': 'Wetland, sparse vegetation - 15% short veg. cover (103)'},
 {'value': '104',
  'color': '#a0b6c9',
  'label': 'Wetland, sparse vegetation - 19% short veg. cover (104)'},
 {'value': '105',
  'color': '#99b3cb',
  'label': 'Wetland, sparse vegetation - 23% short veg. cover (105)'},
 {'value': '106',
  'color': '#91b1cd',
  'label': 'Wetland, sparse vegetation - 27% short veg. cover (106)'},
 {'value': '107',
  'color': '#89afd0',
  'label': 'Wetland, sparse vegetation - 31% short veg. cover (107)'},
 {'value': '108',
  'color': '#82acd2',
  'label': 'Wetland, sparse vegetation - 35% short veg. cover (108)'},
 {'value': '109',
  'color': '#7aaad4',
  'label': 'Wetland, sparse vegetation - 39% short veg. cover (109)'},
 {'value': '110',
  'color': '#73a7d6',
  'label': 'Wetland, sparse vegetation - 43% short veg. cover (110)'},
 {'value': '111',
  'color': '#6ba5d9',
  'label': 'Wetland, sparse vegetation - 47% short veg. cover (111)'},
 {'value': '112',
  'color': '#64a3db',
  'label': 'Wetland, sparse vegetation - 51% short veg. cover (112)'},
 {'value': '113',
  'color': '#5ca0dd',
  'label': 'Wetland, sparse vegetation - 55% short veg. cover (113)'},
 {'value': '114',
  'color': '#549ee0',
  'label': 'Wetland, sparse vegetation - 59% short veg. cover (114)'},
 {'value': '115',
  'color': '#4d9be2',
  'label': 'Wetland, sparse vegetation - 63% short veg. cover (115)'},
 {'value': '116',
  'color': '#4599e4',
  'label': 'Wetland, sparse vegetation - 67% short veg. cover (116)'},
 {'value': '117',
  'color': '#3e96e6',
  'label': 'Wetland, sparse vegetation - 71% short veg. cover (117)'},
 {'value': '118',
  'color': '#3694e9',
  'label': 'Wetland, sparse vegetation - 75% short veg. cover (118)'},
 {'value': '119',
  'color': '#2e92eb',
  'label': 'Wetland, dense short vegetation - 79% short veg. cover (119)'},
 {'value': '120',
  'color': '#278fed',
  'label': 'Wetland, dense short vegetation - 83% short veg. cover (120)'},
 {'value': '121',
  'color': '#1f8df0',
  'label': 'Wetland, dense short vegetation - 87% short veg. cover (121)'},
 {'value': '122',
  'color': '#188af2',
  'label': 'Wetland, dense short vegetation - 91% short veg. cover (122)'},
 {'value': '123',
  'color': '#1088f4',
  'label': 'Wetland, dense short vegetation - 95% short veg. cover (123)'},
 {'value': '124',
  'color': '#0986f7',
  'label': 'Wetland, dense short vegetation - 100% short veg. cover (124)'},
 {'value': '125',
  'color': '#55a5a5',
  'label': 'Wetland, stable tree cover - 3m trees (125)'},
 {'value': '126',
  'color': '#53a1a2',
  'label': 'Wetland, stable tree cover - 4m trees (126)'},
 {'value': '127',
  'color': '#519e9f',
  'label': 'Wetland, stable tree cover - 5m trees (127)'},
 {'value': '128',
  'color': '#4f9b9c',
  'label': 'Wetland, stable tree cover - 6m trees (128)'},
 {'value': '129',
  'color': '#4d989a',
  'label': 'Wetland, stable tree cover - 7m trees (129)'},
 {'value': '130',
  'color': '#4b9597',
  'label': 'Wetland, stable tree cover - 8m trees (130)'},
 {'value': '131',
  'color': '#499294',
  'label': 'Wetland, stable tree cover - 9m trees (131)'},
 {'value': '132',
  'color': '#478f91',
  'label': 'Wetland, stable tree cover - 10m trees (132)'},
 {'value': '133',
  'color': '#458b8f',
  'label': 'Wetland, stable tree cover - 11m trees (133)'},
 {'value': '134',
  'color': '#43888c',
  'label': 'Wetland, stable tree cover - 12m trees (134)'},
 {'value': '135',
  'color': '#418589',
  'label': 'Wetland, stable tree cover - 13m trees (135)'},
 {'value': '136',
  'color': '#3f8286',
  'label': 'Wetland, stable tree cover - 14m trees (136)'},
 {'value': '137',
  'color': '#3d7f84',
  'label': 'Wetland, stable tree cover - 15m trees (137)'},
 {'value': '138',
  'color': '#3b7c81',
  'label': 'Wetland, stable tree cover - 16m trees (138)'},
 {'value': '139',
  'color': '#39797e',
  'label': 'Wetland, stable tree cover - 17m trees (139)'},
 {'value': '140',
  'color': '#37767b',
  'label': 'Wetland, stable tree cover - 18m trees (140)'},
 {'value': '141',
  'color': '#357279',
  'label': 'Wetland, stable tree cover - 19m trees (141)'},
 {'value': '142',
  'color': '#336f76',
  'label': 'Wetland, stable tree cover - 20m trees (142)'},
 {'value': '143',
  'color': '#316c73',
  'label': 'Wetland, stable tree cover - 21m trees (143)'},
 {'value': '144',
  'color': '#2f6970',
  'label': 'Wetland, stable tree cover - 22m trees (144)'},
 {'value': '145',
  'color': '#2d666e',
  'label': 'Wetland, stable tree cover - 23m trees (145)'},
 {'value': '146',
  'color': '#2b636b',
  'label': 'Wetland, stable tree cover - 24m trees (146)'},
 {'value': '147',
  'color': '#296068',
  'label': 'Wetland, stable tree cover - 25m trees (147)'},
 {'value': '148',
  'color': '#285d66',
  'label': 'Wetland, stable tree cover - >25m trees (148)'},
 {'value': '149',
  'color': '#bb93b0',
  'label': 'Wetland, tree cover with prev. disturb. (2020 height) - 3m trees (149)'},
 {'value': '150',
  'color': '#b78fac',
  'label': 'Wetland, tree cover with prev. disturb. (2020 height) - 4m trees (150)'},
 {'value': '151',
  'color': '#b48ca9',
  'label': 'Wetland, tree cover with prev. disturb. (2020 height) - 5m trees (151)'},
 {'value': '152',
  'color': '#b189a6',
  'label': 'Wetland, tree cover with prev. disturb. (2020 height) - 6m trees (152)'},
 {'value': '153',
  'color': '#ae85a2',
  'label': 'Wetland, tree cover with prev. disturb. (2020 height) - 7m trees (153)'},
 {'value': '154',
  'color': '#aa829f',
  'label': 'Wetland, tree cover with prev. disturb. (2020 height) - 8m trees (154)'},
 {'value': '155',
  'color': '#a77f9c',
  'label': 'Wetland, tree cover with prev. disturb. (2020 height) - 9m trees (155)'},
 {'value': '156',
  'color': '#a47b99',
  'label': 'Wetland, tree cover with prev. disturb. (2020 height) - 10m trees (156)'},
 {'value': '157',
  'color': '#a17895',
  'label': 'Wetland, tree cover with prev. disturb. (2020 height) - 11m trees (157)'},
 {'value': '158',
  'color': '#9e7592',
  'label': 'Wetland, tree cover with prev. disturb. (2020 height) - 12m trees (158)'},
 {'value': '159',
  'color': '#9a718f',
  'label': 'Wetland, tree cover with prev. disturb. (2020 height) - 13m trees (159)'},
 {'value': '160',
  'color': '#976e8c',
  'label': 'Wetland, tree cover with prev. disturb. (2020 height) - 14m trees (160)'},
 {'value': '161',
  'color': '#946b88',
  'label': 'Wetland, tree cover with prev. disturb. (2020 height) - 15m trees (161)'},
 {'value': '162',
  'color': '#916885',
  'label': 'Wetland, tree cover with prev. disturb. (2020 height) - 16m trees (162)'},
 {'value': '163',
  'color': '#8d6482',
  'label': 'Wetland, tree cover with prev. disturb. (2020 height) - 17m trees (163)'},
 {'value': '164',
  'color': '#8a617f',
  'label': 'Wetland, tree cover with prev. disturb. (2020 height) - 18m trees (164)'},
 {'value': '165',
  'color': '#875e7b',
  'label': 'Wetland, tree cover with prev. disturb. (2020 height) - 19m trees (165)'},
 {'value': '166',
  'color': '#845a78',
  'label': 'Wetland, tree cover with prev. disturb. (2020 height) - 20m trees (166)'},
 {'value': '167',
  'color': '#815775',
  'label': 'Wetland, tree cover with prev. disturb. (2020 height) - 21m trees (167)'},
 {'value': '168',
  'color': '#7d5472',
  'label': 'Wetland, tree cover with prev. disturb. (2020 height) - 22m trees (168)'},
 {'value': '169',
  'color': '#7a506e',
  'label': 'Wetland, tree cover with prev. disturb. (2020 height) - 23m trees (169)'},
 {'value': '170',
  'color': '#774d6b',
  'label': 'Wetland, tree cover with prev. disturb. (2020 height) - 24m trees (170)'},
 {'value': '171',
  'color': '#744a68',
  'label': 'Wetland, tree cover with prev. disturb. (2020 height) - 25m trees (171)'},
 {'value': '172',
  'color': '#714765',
  'label': 'Wetland, tree cover with prev. disturb. (2020 height) - >25m trees (172)'},
 {'value': '173',
  'color': '#de7cbb',
  'label': 'Wetland, tree height gain (2020 height) - 3m trees (173)'},
 {'value': '174',
  'color': '#da77b7',
  'label': 'Wetland, tree height gain (2020 height) - 4m trees (174)'},
 {'value': '175',
  'color': '#d772b3',
  'label': 'Wetland, tree height gain (2020 height) - 5m trees (175)'},
 {'value': '176',
  'color': '#d46eaf',
  'label': 'Wetland, tree height gain (2020 height) - 6m trees (176)'},
 {'value': '177',
  'color': '#d169ab',
  'label': 'Wetland, tree height gain (2020 height) - 7m trees (177)'},
 {'value': '178',
  'color': '#ce64a8',
  'label': 'Wetland, tree height gain (2020 height) - 8m trees (178)'},
 {'value': '179',
  'color': '#cb60a4',
  'label': 'Wetland, tree height gain (2020 height) - 9m trees (179)'},
 {'value': '180',
  'color': '#c85ba0',
  'label': 'Wetland, tree height gain (2020 height) - 10m trees (180)'},
 {'value': '181',
  'color': '#c4579c',
  'label': 'Wetland, tree height gain (2020 height) - 11m trees (181)'},
 {'value': '182',
  'color': '#c15298',
  'label': 'Wetland, tree height gain (2020 height) - 12m trees (182)'},
 {'value': '183',
  'color': '#be4d95',
  'label': 'Wetland, tree height gain (2020 height) - 13m trees (183)'},
 {'value': '184',
  'color': '#bb4991',
  'label': 'Wetland, tree height gain (2020 height) - 14m trees (184)'},
 {'value': '185',
  'color': '#b8448d',
  'label': 'Wetland, tree height gain (2020 height) - 15m trees (185)'},
 {'value': '186',
  'color': '#b54089',
  'label': 'Wetland, tree height gain (2020 height) - 16m trees (186)'},
 {'value': '187',
  'color': '#b23b86',
  'label': 'Wetland, tree height gain (2020 height) - 17m trees (187)'},
 {'value': '188',
  'color': '#af3682',
  'label': 'Wetland, tree height gain (2020 height) - 18m trees (188)'},
 {'value': '189',
  'color': '#ab327e',
  'label': 'Wetland, tree height gain (2020 height) - 19m trees (189)'},
 {'value': '190',
  'color': '#a82d7a',
  'label': 'Wetland, tree height gain (2020 height) - 20m trees (190)'},
 {'value': '191',
  'color': '#a52976',
  'label': 'Wetland, tree height gain (2020 height) - 21m trees (191)'},
 {'value': '192',
  'color': '#a22473',
  'label': 'Wetland, tree height gain (2020 height) - 22m trees (192)'},
 {'value': '193',
  'color': '#9f1f6f',
  'label': 'Wetland, tree height gain (2020 height) - 23m trees (193)'},
 {'value': '194',
  'color': '#9c1b6b',
  'label': 'Wetland, tree height gain (2020 height) - 24m trees (194)'},
 {'value': '195',
  'color': '#991667',
  'label': 'Wetland, tree height gain (2020 height) - 25m trees (195)'},
 {'value': '196',
  'color': '#961264',
  'label': 'Wetland, tree height gain (2020 height) - >25m trees (196)'},
 {'value': '208',
  'color': '#040464',
  'label': 'Open surface water, permanent water (208)'},
 {'value': '209',
  'color': '#0000ff',
  'label': 'Open surface water, persistent water loss (209)'},
 {'value': '210',
  'color': '#000000',
  'label': 'Open surface water, persistent water gain (210)'},
 {'value': '211',
  'color': '#000000',
  'label': 'Open surface water, variable water (211)'},
 {'value': '240',
  'color': '#ff2828',
  'label': 'Short veg. after tree loss (240)'},
 {'value': '241', 'color': '#ffffff', 'label': 'Snow/ice, stable (241)'},
 {'value': '242', 'color': '#d0ffff', 'label': 'Snow/ice, gain (242)'},
 {'value': '243', 'color': '#ffe0d0', 'label': 'Snow/ice, loss (243)'},
 {'value': '244', 'color': '#ff7d00', 'label': 'Cropland, Stable (244)'},
 {'value': '245',
  'color': '#ffff00',
  'label': 'Cropland, gain from trees (245)'},
 {'value': '246',
  'color': '#c86400',
  'label': 'Cropland, gain from wetland veg (246)'},
 {'value': '247',
  'color': '#fac800',
  'label': 'Cropland, gain from other (247)'},
 {'value': '248', 'color': '#afcd96', 'label': 'Cropland, loss to tree (248)'},
 {'value': '249',
  'color': '#afcd96',
  'label': 'Cropland, loss to short veg/other (249)'},
 {'value': '250',
  'color': '#64dcdc',
  'label': 'Built-up, stable built-up (250)'},
 {'value': '251',
  'color': '#00ffff',
  'label': 'Built-up, gain from trees (251)'},
 {'value': '252',
  'color': '#00ffff',
  'label': 'Built-up, gain from crop (252)'},
 {'value': '253',
  'color': '#00ffff',
  'label': 'Built-up, gain from other (253)'},
 {'value': '254', 'color': '#111133', 'label': 'Ocean (254)'},
 {'value': '255', 'color': '#000000', 'label': 'No data (255)'}]
colormap = {str(x["value"]): x["color"] for x in legend}
legend_short = legend = [
 {'value': '1',
  'color': '#fafac3',
  'label': 'Terra Firma, true desert - 7% short veg. cover (1)'},
  {'value': '24', 'color': '#b0b006',
  'label': 'Terra Firma, dense short vegetation - 100% short veg. cover (24)'},
 {'value': '25',
  'color': '#609c60',
  'label': 'Terra Firma, stable tree cover - 3m trees (25)'},
 {'value': '48',
  'color': '#065106',
  'label': 'Terra Firma, stable tree cover - >25m trees (48)'},
 {'value': '49',
  'color': '#643700',
  'label': 'Terra Firma, tree cover with prev. disturb. (2020 height) - 3m trees (49)'},
 {'value': '72',
  'color': '#667b00',
  'label': 'Terra Firma, tree cover with prev. disturb. (2020 height) - >25m trees (72)'},
 {'value': '73',
  'color': '#ff99ff',
  'label': 'Terra Firma, tree height gain (2020 height) - 3m trees (73)'},
 {'value': '96',
  'color': '#bb00bb',
  'label': 'Terra Firma, tree height gain (2020 height) - >25m trees (96)'},
 {'value': '100',
  'color': '#bfc0c0',
  'label': 'Wetland, salt pan - 3% short veg. cover (100)'},
 {'value': '124',
  'color': '#0986f7',
  'label': 'Wetland, dense short vegetation - 100% short veg. cover (124)'},
 {'value': '125',
  'color': '#55a5a5',
  'label': 'Wetland, stable tree cover - 3m trees (125)'},
 {'value': '148',
  'color': '#285d66',
  'label': 'Wetland, stable tree cover - >25m trees (148)'},
 {'value': '149',
  'color': '#bb93b0',
  'label': 'Wetland, tree cover with prev. disturb. (2020 height) - 3m trees (149)'},
 {'value': '172',
  'color': '#714765',
  'label': 'Wetland, tree cover with prev. disturb. (2020 height) - >25m trees (172)'},
 {'value': '173',
  'color': '#de7cbb',
  'label': 'Wetland, tree height gain (2020 height) - 3m trees (173)'},
 {'value': '196',
  'color': '#961264',
  'label': 'Wetland, tree height gain (2020 height) - >25m trees (196)'},
 {'value': '208',
  'color': '#040464',
  'label': 'Open surface water, permanent water (208)'},
 {'value': '209',
  'color': '#0000ff',
  'label': 'Open surface water, persistent water loss (209)'},
 {'value': '210',
  'color': '#000000',
  'label': 'Open surface water, persistent water gain (210)'},
 {'value': '211',
  'color': '#000000',
  'label': 'Open surface water, variable water (211)'},
 {'value': '240',
  'color': '#ff2828',
  'label': 'Short veg. after tree loss (240)'},
 {'value': '241', 'color': '#ffffff', 'label': 'Snow/ice, stable (241)'},
 {'value': '242', 'color': '#d0ffff', 'label': 'Snow/ice, gain (242)'},
 {'value': '243', 'color': '#ffe0d0', 'label': 'Snow/ice, loss (243)'},
 {'value': '244', 'color': '#ff7d00', 'label': 'Cropland, Stable (244)'},
 {'value': '245',
  'color': '#ffff00',
  'label': 'Cropland, gain from trees (245)'},
 {'value': '246',
  'color': '#c86400',
  'label': 'Cropland, gain from wetland veg (246)'},
 {'value': '247',
  'color': '#fac800',
  'label': 'Cropland, gain from other (247)'},
 {'value': '248', 'color': '#afcd96', 'label': 'Cropland, loss to tree (248)'},
 {'value': '249',
  'color': '#afcd96',
  'label': 'Cropland, loss to short veg/other (249)'},
 {'value': '250',
  'color': '#64dcdc',
  'label': 'Built-up, stable built-up (250)'},
 {'value': '251',
  'color': '#00ffff',
  'label': 'Built-up, gain from trees (251)'},
 {'value': '252',
  'color': '#00ffff',
  'label': 'Built-up, gain from crop (252)'},
 {'value': '253',
  'color': '#00ffff',
  'label': 'Built-up, gain from other (253)'},
 {'value': '254', 'color': '#111133', 'label': 'Ocean (254)'},
 {'value': '255', 'color': '#000000', 'label': 'No data (255)'}]
for year, cog in lc_glad.items():
  m.add_cog_layer(
    cog,
    colormap=json.dumps(colormap),
    name=year,
    attribution="UMD GLAD",
    shown=True
  )
m.add_geojson(geometries, layer_name="Your Parcels", zoom_to_layer=True,)

m.add_legend(
  labels=[x["label"] for x in legend_short],
  colors=[x["color"] for x in legend_short],
  position="bottomright",
  title="Legend",
)
leafmap.create_legend(
  title="Legend",
  labels=[x["label"] for x in legend_short],
  colors=[x["color"] for x in legend_short],
  position="bottomright",
)
# m.add_inspector_gui()
# m.fit_bounds(bounds_list)
# m.zoom_to_bounds(bounds_list)
m

:::


## Above Ground Biomass

In [ ]:
m = leafmap.Map(
    center=(center[1], center[0]),
    zoom=12,
    draw_control=False,
    measure_control=False,
    fullscreen_control=False,
    attribution_control=False,
    search_control=False,
    layers_control=True,
    scale_control=False,
    toolbar_control=False,
    basemap="Esri.WorldGrayCanvas"
)
m.add_tile_layer(
  url="https://storage.googleapis.com/esa-cfs-tiles/1.16.2/biomass.agb/tiles/1/{x}/{y}/{z}.png",
  name="Above Ground Biomass",
  attribution="ESA CCI Biomass",
)
m

## Soil Properties
> Soil properties influence water retention, erosion, and crop growth. Understanding texture, organic matter, and compaction help to reduce risks, make informed decisions, and improve long-term soil health.

### Soil Taxonomy


In [ ]:
from IPython.display import HTML
usda_class_cog = "https://s3.openlandmap.org/arco/grtgroup_usda.soiltax_c_250m_s_19500101_20171231_go_espg.4326_v0.2.tif"
legend = [{'value': '0', 'label': 'NODATA', 'color': '#FFFFFF'},
 {'value': '1', 'label': 'Albaqualfs', 'color': '#ADFF2D'},
 {'value': '2', 'label': 'Cryaqualfs', 'color': '#ADFF22'},
 {'value': '4', 'label': 'Durixeralfs', 'color': '#A5FF2F'},
 {'value': '6', 'label': 'Endoaqualfs', 'color': '#87FF37'},
 {'value': '7', 'label': 'Epiaqualfs', 'color': '#BAF019'},
 {'value': '9', 'label': 'Fragiaqualfs', 'color': '#87FF19'},
 {'value': '10', 'label': 'Fragiudalfs', 'color': '#96F03D'},
 {'value': '11', 'label': 'Fragixeralfs', 'color': '#A3F52F'},
 {'value': '12', 'label': 'Fraglossudalfs', 'color': '#AFF319'},
 {'value': '13', 'label': 'Glossaqualfs', 'color': '#91FF37'},
 {'value': '14', 'label': 'Glossocryalfs', 'color': '#9CF319'},
 {'value': '15', 'label': 'Glossudalfs', 'color': '#9BFF37'},
 {'value': '16', 'label': 'Haplocryalfs', 'color': '#91FF19'},
 {'value': '17', 'label': 'Haploxeralfs', 'color': '#71FF37'},
 {'value': '18', 'label': 'Hapludalfs', 'color': '#86FF19'},
 {'value': '19', 'label': 'Haplustalfs', 'color': '#A9D42D'},
 {'value': '25', 'label': 'Natraqualfs', 'color': '#AFF519'},
 {'value': '26', 'label': 'Natrixeralfs', 'color': '#9BFF19'},
 {'value': '27', 'label': 'Natrudalfs', 'color': '#9AF024'},
 {'value': '28', 'label': 'Natrustalfs', 'color': '#A5FD2F'},
 {'value': '29', 'label': 'Palecryalfs', 'color': '#88FF37'},
 {'value': '30', 'label': 'Paleudalfs', 'color': '#AFED19'},
 {'value': '31', 'label': 'Paleustalfs', 'color': '#71FF19'},
 {'value': '32', 'label': 'Palexeralfs', 'color': '#AFF026'},
 {'value': '38', 'label': 'Rhodustalfs', 'color': '#8CF537'},
 {'value': '39', 'label': 'Vermaqualfs', 'color': '#B7FF19'},
 {'value': '41', 'label': 'Eutroboralfs', 'color': '#7177C0'},
 {'value': '42', 'label': 'Ochraqualfs', 'color': '#9A85EC'},
 {'value': '43', 'label': 'Glossoboralfs', 'color': '#F5F5E1'},
 {'value': '44', 'label': 'Cryoboralfs', 'color': '#52CF5A'},
 {'value': '45', 'label': 'Natriboralfs', 'color': '#E42777'},
 {'value': '46', 'label': 'Paleboralfs', 'color': '#4EF76D'},
 {'value': '50', 'label': 'Cryaquands', 'color': '#FF00FB'},
 {'value': '58', 'label': 'Fulvicryands', 'color': '#EB05EB'},
 {'value': '59', 'label': 'Fulvudands', 'color': '#FA04FA'},
 {'value': '61', 'label': 'Haplocryands', 'color': '#FC04F5'},
 {'value': '63', 'label': 'Haploxerands', 'color': '#F50DF0'},
 {'value': '64', 'label': 'Hapludands', 'color': '#F118F1'},
 {'value': '74', 'label': 'Udivitrands', 'color': '#FA0CFA'},
 {'value': '75', 'label': 'Ustivitrands', 'color': '#FC05E1'},
 {'value': '76', 'label': 'Vitraquands', 'color': '#F100D5'},
 {'value': '77', 'label': 'Vitricryands', 'color': '#EB09E6'},
 {'value': '80', 'label': 'Vitrixerands', 'color': '#FA22FA'},
 {'value': '82', 'label': 'Aquicambids', 'color': '#FFDAB9'},
 {'value': '83', 'label': 'Aquisalids', 'color': '#F5D2BB'},
 {'value': '85', 'label': 'Argidurids', 'color': '#E8C9B8'},
 {'value': '86', 'label': 'Argigypsids', 'color': '#FFDDC4'},
 {'value': '87', 'label': 'Calciargids', 'color': '#E7CBC0'},
 {'value': '89', 'label': 'Calcigypsids', 'color': '#FFD2C3'},
 {'value': '90', 'label': 'Gypsiargids', 'color': '#F5D6BB'},
 {'value': '92', 'label': 'Haplargids', 'color': '#D5D3B9'},
 {'value': '93', 'label': 'Haplocalcids', 'color': '#E8D4B8'},
 {'value': '94', 'label': 'Haplocambids', 'color': '#E7CDC0'},
 {'value': '96', 'label': 'Haplodurids', 'color': '#F3EAC8'},
 {'value': '97', 'label': 'Haplogypsids', 'color': '#A0C4BA'},
 {'value': '98', 'label': 'Haplosalids', 'color': '#FFD2B9'},
 {'value': '99', 'label': 'Natrargids', 'color': '#F5DABB'},
 {'value': '100', 'label': 'Natridurids', 'color': '#F5D5B9'},
 {'value': '101', 'label': 'Natrigypsids', 'color': '#E8EBB8'},
 {'value': '102', 'label': 'Paleargids', 'color': '#FFDDC2'},
 {'value': '103', 'label': 'Petroargids', 'color': '#E7FFC0'},
 {'value': '104', 'label': 'Petrocalcids', 'color': '#F3E6C8'},
 {'value': '105', 'label': 'Petrocambids', 'color': '#FFDAB9'},
 {'value': '107', 'label': 'Petrogypsids', 'color': '#F5CDB9'},
 {'value': '110', 'label': 'Calciorthids', 'color': '#A91D30'},
 {'value': '111', 'label': 'Camborthids', 'color': '#796578'},
 {'value': '112', 'label': 'Paleorthids', 'color': '#D8FF6E'},
 {'value': '113', 'label': 'Durorthids', 'color': '#177548'},
 {'value': '114', 'label': 'Durargids', 'color': '#43EFD6'},
 {'value': '115', 'label': 'Gypsiorthids', 'color': '#8496A9'},
 {'value': '116', 'label': 'Nadurargids', 'color': '#296819'},
 {'value': '118', 'label': 'Cryaquents', 'color': '#73FFD4'},
 {'value': '119', 'label': 'Cryofluvents', 'color': '#6FFFC8'},
 {'value': '120', 'label': 'Cryopsamments', 'color': '#75FBC9'},
 {'value': '121', 'label': 'Cryorthents', 'color': '#86F5D1'},
 {'value': '122', 'label': 'Endoaquents', 'color': '#82FFD2'},
 {'value': '123', 'label': 'Epiaquents', 'color': '#88EEC8'},
 {'value': '124', 'label': 'Fluvaquents', 'color': '#80FFD4'},
 {'value': '126', 'label': 'Frasiwassents', 'color': '#6BFFC9'},
 {'value': '131', 'label': 'Hydraquents', 'color': '#88EEC8'},
 {'value': '133', 'label': 'Psammaquents', 'color': '#7FFFC8'},
 {'value': '134', 'label': 'Psammowassents', 'color': '#81FFD2'},
 {'value': '135', 'label': 'Quartzipsamments', 'color': '#86F0D4'},
 {'value': '136', 'label': 'Sulfaquents', 'color': '#67FFC8'},
 {'value': '137', 'label': 'Sulfiwassents', 'color': '#88EEC8'},
 {'value': '138', 'label': 'Torrifluvents', 'color': '#7FFBCB'},
 {'value': '139', 'label': 'Torriorthents', 'color': '#87FFD2'},
 {'value': '140', 'label': 'Torripsamments', 'color': '#8AF5CE'},
 {'value': '141', 'label': 'Udifluvents', 'color': '#6BFAD2'},
 {'value': '142', 'label': 'Udipsamments', 'color': '#78F0D4', "description": "Udipsamments are sandy soils in the Entisols order, known for minimal profile development and good drainage. Found in areas like coastal dunes and river terraces, they have low fertility but support vegetation adapted to well-drained conditions. The name reflects their moist climate and sandy texture."},
 {'value': '143', 'label': 'Udorthents', 'color': '#88EEC8'},
 {'value': '144', 'label': 'Ustifluvents', 'color': '#7FFBD4'},
 {'value': '145', 'label': 'Ustipsamments', 'color': '#73F5CD'},
 {'value': '146', 'label': 'Ustorthents', 'color': '#88C8D2'},
 {'value': '147', 'label': 'Xerofluvents', 'color': '#91F0CD'},
 {'value': '148', 'label': 'Xeropsamments', 'color': '#73CDD2'},
 {'value': '149', 'label': 'Xerorthents', 'color': '#88EEC8'},
 {'value': '153', 'label': 'Udarents', 'color': '#FB849B'},
 {'value': '154', 'label': 'Torriarents', 'color': '#DD4479'},
 {'value': '155', 'label': 'Xerarents', 'color': '#61388B'},
 {'value': '179', 'label': 'Cryofibrists', 'color': '#A52A30'},
 {'value': '180', 'label': 'Cryofolists', 'color': '#722328'},
 {'value': '181', 'label': 'Cryohemists', 'color': '#D81419'},
 {'value': '182', 'label': 'Cryosaprists', 'color': '#A42828'},
 {'value': '183', 'label': 'Frasiwassists', 'color': '#82F5CD'},
 {'value': '184', 'label': 'Haplofibrists', 'color': '#A54C2E'},
 {'value': '185', 'label': 'Haplohemists', 'color': '#C11919'},
 {'value': '186', 'label': 'Haplosaprists', 'color': '#B91419'},
 {'value': '189', 'label': 'Sphagnofibrists', 'color': '#21B199'},
 {'value': '190', 'label': 'Sulfihemists', 'color': '#702028'},
 {'value': '191', 'label': 'Sulfisaprists', 'color': '#B41919'},
 {'value': '196', 'label': 'Udifolists', 'color': '#B22328'},
 {'value': '201', 'label': 'Borosaprists', 'color': '#A2C7EB'},
 {'value': '202', 'label': 'Medisaprists', 'color': '#36BA79'},
 {'value': '203', 'label': 'Borohemists', 'color': '#806797'},
 {'value': '206', 'label': 'Calcicryepts', 'color': '#CB5B5F'},
 {'value': '207', 'label': 'Calciustepts', 'color': '#CD5C5C'},
 {'value': '208', 'label': 'Calcixerepts', 'color': '#D94335'},
 {'value': '209', 'label': 'Cryaquepts', 'color': '#D35740'},
 {'value': '210', 'label': 'Durixerepts', 'color': '#E05A5D'},
 {'value': '212', 'label': 'Durustepts', 'color': '#CF5B5C'},
 {'value': '213', 'label': 'Dystrocryepts', 'color': '#CA5964'},
 {'value': '215', 'label': 'Dystroxerepts', 'color': '#CA5D5F'},
 {'value': '216', 'label': 'Dystrudepts', 'color': '#CD5E5A'},
 {'value': '217', 'label': 'Dystrustepts', 'color': '#CA5969'},
 {'value': '218', 'label': 'Endoaquepts', 'color': '#D95A35'},
 {'value': '219', 'label': 'Epiaquepts', 'color': '#D36240'},
 {'value': '220', 'label': 'Eutrudepts', 'color': '#E05C43'},
 {'value': '221', 'label': 'Fragiaquepts', 'color': '#D64755'},
 {'value': '222', 'label': 'Fragiudepts', 'color': '#CF595C'},
 {'value': '225', 'label': 'Halaquepts', 'color': '#FF5F5F'},
 {'value': '226', 'label': 'Haplocryepts', 'color': '#CD6058'},
 {'value': '228', 'label': 'Haploxerepts', 'color': '#D95F35'},
 {'value': '229', 'label': 'Haplustepts', 'color': '#D35140'},
 {'value': '230', 'label': 'Humaquepts', 'color': '#D65A55'},
 {'value': '231', 'label': 'Humicryepts', 'color': '#E05C59'},
 {'value': '233', 'label': 'Humixerepts', 'color': '#CF525E'},
 {'value': '234', 'label': 'Humudepts', 'color': '#C65978'},
 {'value': '235', 'label': 'Humustepts', 'color': '#F5615F'},
 {'value': '245', 'label': 'Ustochrepts', 'color': '#826F9A'},
 {'value': '246', 'label': 'Eutrochrepts', 'color': '#CFF41A'},
 {'value': '247', 'label': 'Dystrochrepts', 'color': '#4A6F31'},
 {'value': '248', 'label': 'Eutrocryepts', 'color': '#A96989'},
 {'value': '249', 'label': 'Haplaquepts', 'color': '#E16438'},
 {'value': '250', 'label': 'Xerochrepts', 'color': '#24F640'},
 {'value': '251', 'label': 'Cryochrepts', 'color': '#88C1F9'},
 {'value': '252', 'label': 'Fragiochrepts', 'color': '#F5D25C'},
 {'value': '253', 'label': 'Haplumbrepts', 'color': '#D74322'},
 {'value': '254', 'label': 'Cryumbrepts', 'color': '#7F939E'},
 {'value': '255', 'label': 'Dystropepts', 'color': '#41A545'},
 {'value': '256', 'label': 'Vitrandepts', 'color': '#8F8340'},
 {'value': '268', 'label': 'Argialbolls', 'color': '#09FE03'},
 {'value': '269', 'label': 'Argiaquolls', 'color': '#0AFF00'},
 {'value': '270', 'label': 'Argicryolls', 'color': '#0FF30F'},
 {'value': '271', 'label': 'Argiudolls', 'color': '#02F00A'},
 {'value': '272', 'label': 'Argiustolls', 'color': '#0FC903'},
 {'value': '273', 'label': 'Argixerolls', 'color': '#17F000'},
 {'value': '274', 'label': 'Calciaquolls', 'color': '#0CFF00'},
 {'value': '275', 'label': 'Calcicryolls', 'color': '#0AC814'},
 {'value': '276', 'label': 'Calciudolls', 'color': '#0CFE00'},
 {'value': '277', 'label': 'Calciustolls', 'color': '#0AFF0A'},
 {'value': '278', 'label': 'Calcixerolls', 'color': '#03FF05'},
 {'value': '279', 'label': 'Cryaquolls', 'color': '#1CF31C'},
 {'value': '280', 'label': 'Cryrendolls', 'color': '#24F000'},
 {'value': '283', 'label': 'Durixerolls', 'color': '#00FF0C'},
 {'value': '284', 'label': 'Durustolls', 'color': '#14C814'},
 {'value': '285', 'label': 'Endoaquolls', 'color': '#00FE4C'},
 {'value': '286', 'label': 'Epiaquolls', 'color': '#14FF96'},
 {'value': '287', 'label': 'Haplocryolls', 'color': '#44D205'},
 {'value': '289', 'label': 'Haploxerolls', 'color': '#05F305'},
 {'value': '290', 'label': 'Hapludolls', 'color': '#62F00A'},
 {'value': '291', 'label': 'Haplustolls', 'color': '#0FCD03'},
 {'value': '292', 'label': 'Haprendolls', 'color': '#00D20F'},
 {'value': '294', 'label': 'Natraquolls', 'color': '#1ADD11'},
 {'value': '296', 'label': 'Natrixerolls', 'color': '#09FF0C'},
 {'value': '297', 'label': 'Natrudolls', 'color': '#03FF05'},
 {'value': '298', 'label': 'Natrustolls', 'color': '#05E700'},
 {'value': '299', 'label': 'Palecryolls', 'color': '#02F00A'},
 {'value': '300', 'label': 'Paleudolls', 'color': '#0FEA03'},
 {'value': '301', 'label': 'Paleustolls', 'color': '#00F000'},
 {'value': '302', 'label': 'Palexerolls', 'color': '#0CCB0C'},
 {'value': '303', 'label': 'Vermudolls', 'color': '#14DD14'},
 {'value': '306', 'label': 'Haploborolls', 'color': '#6A685D'},
 {'value': '307', 'label': 'Argiborolls', 'color': '#FAE6B9'},
 {'value': '308', 'label': 'Haplaquolls', 'color': '#769A34'},
 {'value': '309', 'label': 'Cryoborolls', 'color': '#6FF2DF'},
 {'value': '310', 'label': 'Natriborolls', 'color': '#CA7FC6'},
 {'value': '311', 'label': 'Calciborolls', 'color': '#D8228F'},
 {'value': '312', 'label': 'Paleborolls', 'color': '#C01BF0'},
 {'value': '342', 'label': 'Alaquods', 'color': '#D2BAD3'},
 {'value': '343', 'label': 'Alorthods', 'color': '#D8C3CB'},
 {'value': '345', 'label': 'Duraquods', 'color': '#D4C6D4'},
 {'value': '348', 'label': 'Durorthods', 'color': '#D5BED5'},
 {'value': '349', 'label': 'Endoaquods', 'color': '#DDB9DD'},
 {'value': '350', 'label': 'Epiaquods', 'color': '#D8D2D8'},
 {'value': '351', 'label': 'Fragiaquods', 'color': '#D4C9D4'},
 {'value': '353', 'label': 'Fragiorthods', 'color': '#D2BAD5'},
 {'value': '354', 'label': 'Haplocryods', 'color': '#D5BAD5'},
 {'value': '356', 'label': 'Haplohumods', 'color': '#D5B2D5'},
 {'value': '357', 'label': 'Haplorthods', 'color': '#D8C8D2'},
 {'value': '358', 'label': 'Humicryods', 'color': '#D4CBD4'},
 {'value': '367', 'label': 'Haplaquods', 'color': '#552638'},
 {'value': '368', 'label': 'Cryorthods', 'color': '#2571EB'},
 {'value': '370', 'label': 'Albaquults', 'color': '#FFA514'},
 {'value': '371', 'label': 'Endoaquults', 'color': '#F3A502'},
 {'value': '372', 'label': 'Epiaquults', 'color': '#FB7B00'},
 {'value': '373', 'label': 'Fragiaquults', 'color': '#F0B405'},
 {'value': '374', 'label': 'Fragiudults', 'color': '#F7A80F'},
 {'value': '375', 'label': 'Haplohumults', 'color': '#FB9113'},
 {'value': '376', 'label': 'Haploxerults', 'color': '#FFA519'},
 {'value': '377', 'label': 'Hapludults', 'color': '#F3A702'},
 {'value': '378', 'label': 'Haplustults', 'color': '#FBBA07'},
 {'value': '381', 'label': 'Kandiudults', 'color': '#F7970F'},
 {'value': '385', 'label': 'Kanhapludults', 'color': '#F3A702'},
 {'value': '387', 'label': 'Paleaquults', 'color': '#FB5A00'},
 {'value': '388', 'label': 'Palehumults', 'color': '#F0C005'},
 {'value': '389', 'label': 'Paleudults', 'color': '#F7810F'},
 {'value': '390', 'label': 'Paleustults', 'color': '#FF9C00'},
 {'value': '391', 'label': 'Palexerults', 'color': '#F3B002'},
 {'value': '396', 'label': 'Rhodudults', 'color': '#F0B005'},
 {'value': '399', 'label': 'Umbraquults', 'color': '#F7980F'},
 {'value': '401', 'label': 'Ochraquults', 'color': '#4D7CFC'},
 {'value': '403', 'label': 'Calciaquerts', 'color': '#FFFF00'},
 {'value': '405', 'label': 'Calciusterts', 'color': '#FAFA05'},
 {'value': '406', 'label': 'Calcixererts', 'color': '#EBEB22'},
 {'value': '409', 'label': 'Dystraquerts', 'color': '#FFFF14'},
 {'value': '410', 'label': 'Dystruderts', 'color': '#F1F10A'},
 {'value': '412', 'label': 'Endoaquerts', 'color': '#FAFA05'},
 {'value': '413', 'label': 'Epiaquerts', 'color': '#EBEB1E'},
 {'value': '414', 'label': 'Gypsitorrerts', 'color': '#F5EB0C'},
 {'value': '415', 'label': 'Gypsiusterts', 'color': '#EEF506'},
 {'value': '417', 'label': 'Haplotorrerts', 'color': '#F1F129'},
 {'value': '418', 'label': 'Haploxererts', 'color': '#FAFA05'},
 {'value': '419', 'label': 'Hapluderts', 'color': '#EBEB0C'},
 {'value': '420', 'label': 'Haplusterts', 'color': '#F5D202'},
 {'value': '422', 'label': 'Natraquerts', 'color': '#FFD700'},
 {'value': '424', 'label': 'Salitorrerts', 'color': '#F1F12B'},
 {'value': '429', 'label': 'Chromusterts', 'color': '#A91FAC'},
 {'value': '430', 'label': 'Pellusterts', 'color': '#2DA468'},
 {'value': '431', 'label': 'Chromoxererts', 'color': '#9A8B71'},
 {'value': '432', 'label': 'Pelluderts', 'color': '#76B989'},
 {'value': '433', 'label': 'Torrerts', 'color': '#713959'}]



usda_class_stats = rasterstats.zonal_stats(geometries, usda_class_cog, stats=["majority"], all_touched=True)
usda_class_stats = pd.DataFrame(usda_class_stats)
usda_class_stats["Field"] = "Field " + (usda_class_stats.index + 1).astype(str)

# asign the soil type to the majority value from the legend
usda_class_stats["Soil Type"] = usda_class_stats["majority"].apply(lambda x: next((item["label"] for item in legend if item["value"] == str(int(x))), "NODATA"))

usda_class_stats["Description"] = usda_class_stats["majority"].apply(lambda x: next((item.get("description") for item in legend if item["value"] == str(int(x))), "NODATA"))
usda_class_stats = usda_class_stats.drop(columns=["majority"])
usda_class_stats.groupby("Soil Type")["Field"].apply(lambda x: ", ".join(x)).reset_index().set_index("Soil Type")

In [ ]:
# create a html which shows the descriotion of the soil type if available
for index, row in usda_class_stats[["Soil Type", "Description"]].drop_duplicates().iterrows():
  if row["Description"] != "NODATA" or row["Description"] != "":
    display(HTML(f"<h4>{row['Soil Type']}</h4><p>{row['Description']}</p>"))

### Soil Organic Carbon (SOC)
> Soil organic carbon (SOC) supports sustainable agriculture by improving soil health and sequestering carbon, but declining SOC increases risks like poor soil structure and reduced productivity. The analysis below shows the SOC content for each field at different depths.


In [ ]:
import rasterstats
import pandas as pd

long_term_organic_carbon = [
  {
    "url": "https://s3.openlandmap.org/arco/organic.carbon_usda.6a1c_m_250m_b0cm_19500101_20171231_go_epsg.4326_v0.2.tif",
    "depth": "0cm - 10cm"
  },
  {
    "url": "https://s3.openlandmap.org/arco/organic.carbon_usda.6a1c_m_250m_b10cm_19500101_20171231_go_epsg.4326_v0.2.tif",
    "depth": "10cm - 30cm"
  },
  {
    "url": "https://s3.openlandmap.org/arco/organic.carbon_usda.6a1c_m_250m_b30cm_19500101_20171231_go_epsg.4326_v0.2.tif",
    "depth": "30cm - 60cm"
  },
  # {
  #   "url": "https://s3.openlandmap.org/arco/organic.carbon_usda.6a1c_m_250m_b60cm_19500101_20171231_go_epsg.4326_v0.2.tif",
  #   "depth": "60cm - 100cm"
  # },
  # {
  #   "url": "https://s3.openlandmap.org/arco/organic.carbon_usda.6a1c_m_250m_b100cm_19500101_20171231_go_epsg.4326_v0.2.tif",
  #   "depth": "100cm - 200cm"
  # },
  # {
  #   "url": "https://s3.openlandmap.org/arco/organic.carbon_usda.6a1c_m_250m_b200cm_19500101_20171231_go_epsg.4326_v0.2.tif",
  #   "depth": "200cm - 200cm"
  # }
]
stats_df = []
for i, soc_cog in enumerate(long_term_organic_carbon):
  stats = rasterstats.zonal_stats(geometries, soc_cog["url"], stats=["mean"], all_touched=True)
  stats = pd.DataFrame(stats)
  stats["depth"] = soc_cog["depth"]
  stats_df.append(stats)

stats_df = pd.concat(stats_df)
stats_df["Field"] = "Field " + (stats_df.index + 1).astype(str)
stats_df["Soil Organic Carbon (in %)"] = round(stats_df["mean"] / 2, 2)
stats_df.drop(columns=["mean"], inplace=True)
stats_df = stats_df.groupby(["Field", "depth"]).mean().unstack([-1])
# sort the columns in the order of depth keep in mind that the depth is a string
stats_df = stats_df.reindex(columns=stats_df.columns.sort_values(ascending=True))
stats_df

### Soil Organic Carbon Map
> The map below shows the long term Soil Organic Carbon (SOC) content worldwide.

::: column-screen

In [ ]:
import leafmap
import json
m = leafmap.Map(
    center=(center[1], center[0]),
    zoom=12,
    draw_control=False,
    measure_control=False,
    fullscreen_control=False,
    attribution_control=False,
    search_control=False,
    layers_control=True,
    scale_control=False,
    toolbar_control=False,
)
colormap = [
    ((0, 1), "#ffffa0"),
    ((1, 2), "#f7fcb9"),
    ((2, 4), "#d9f0a3"),
    ((4, 6), "#addd8e"),
    ((6, 10), "#78c679"),
    ((10, 15), "#41ab5d"),
    ((15, 25), "#238443"),
    ((25, 40), "#005b29"),
    ((40, 60), "#004b29"),
    ((60, 120), "#012b13"),
    ((120, 256), "#00120b"),
]
# divide the colormap by 2 to get the percentage
for i in range(len(colormap)):
    colormap[i] = ((colormap[i][0][0] / 2, colormap[i][0][1] / 2), colormap[i][1])

for soc_cog in long_term_organic_carbon:
  m.add_cog_layer(
    soc_cog["url"],
    colormap=json.dumps(colormap),
    name=soc_cog["depth"],
    attribution="OpenLandMap",
  )
m.add_geojson(geometries, layer_name="Your Parcels", zoom_to_layer=True,)
m.add_colorbar(
  colors=[x[1] for x in colormap],
  index=[x[0][0] for x in colormap],
  vmin=colormap[0][0][0],
  vmax=colormap[-1][0][0],
  caption="Soil Organic Carbon (%)")
m

:::

## Biodiversity
### Endangered Species
:::: {.columns}
::: {.column}
The IUCN Red List is a global list that categorizes different species based on their risk of extinction. The goal is to assess the conservation status of species and raise awareness about the dangers they face. It is run by the International Union for Conservation of Nature (IUCN).
:::
::: {.column}
Categories of the IUCN Red List:

- Extinct (EX): No known individuals of the species are left.
-	Extinct in the Wild (EW): Only survives in captivity or cultivation.
-	Critically Endangered (CR): Faces an extremely high risk of extinction in the wild.
-	Endangered (EN): Faces a very high risk of extinction.
-	Vulnerable (VU): Faces a high risk of extinction.
-	Near Threatened (NT): Close to qualifying for a threatened category.
-	Least Concern (LC): Low risk of extinction.
-	Data Deficient (DD): Not enough information to assess its risk.
-	Not Evaluated (NE): Has not been assessed yet.
:::
::::


In [ ]:
from pygbif import occurrences
from shapely.geometry import Polygon, Point
from shapely.geometry.polygon import orient
import geopandas as gpd
from pyproj import Geod


center_point = Point(center)
center_point_gdf = gpd.GeoDataFrame(geometry=[center_point], crs="EPSG:4326").to_crs("EPSG:3857")

center_point_gdf["geometry"] = center_point_gdf.buffer(30000).to_crs("EPSG:4326")
center_point_gdf["geometry"] = center_point_gdf["geometry"].apply(lambda geom: orient(geom, sign=1.0))

center_point_wkt = center_point_gdf["geometry"].to_wkt()[0]

results = occurrences.search(
    limit=10000,
    geometry=center_point_wkt
)

gdbf_df = pd.DataFrame(results["results"])
gdbf_df["geometry"] = gdbf_df.apply(lambda x: Point(x["decimalLongitude"], x["decimalLatitude"]), axis=1)
gdbf_df = gpd.GeoDataFrame(gdbf_df, geometry="geometry", crs="EPSG:4326")

import plotly.express as px

pd.set_option('display.max_columns', None)
iucnRedListCategory = {
    "EX": "Extinct",
    "EW": "Extinct in the Wild",
    "CR": "Critically Endangered",
    "EN": "Endangered",
    "VU": "Vulnerable",
    "NT": "Near Threatened",
    "LC": "Least Concern",
    "DD": "Data Deficient",
    "NE": "Not Evaluated"
}
gdbf_df["iucnRedListCategory"] = gdbf_df["iucnRedListCategory"].replace(
    iucnRedListCategory
)

count = gdbf_df[["kingdom", "class", "species", "iucnRedListCategory"]].drop_duplicates()
count = count.dropna()
count = count[count["iucnRedListCategory"].isin([
    "Extinct",
    "Extinct in the Wild",
    "Critically Endangered",
    "Endangered",
    "Vulnerable",
    "Near Threatened",
    # "Least Concern",
    "Data Deficient"
    "Not Evaluated"
])]
color_map = {
    "Extinct": "#000000",  # Black
    "Extinct in the Wild": "#542344",  # Dark Purple
    "Critically Endangered": "#D81E05",  # Red
    "Endangered": "#FC7F3F",  # Orange
    "Vulnerable": "#F9E814",  # Yellow
    "Near Threatened": "#CCE226",  # Light Green
    "Least Concern": "#60C659",  # Green
    "Data Deficient": "#D1D1C6",  # Light Grey
    "Not Evaluated": "#FFFFFF",  # White
    "Regionally Extinct": "#9B4F96"  # Purple
}

fig = px.treemap(
    count, 
    path=['iucnRedListCategory', "kingdom", "class", "species"],
    color='iucnRedListCategory',
    color_discrete_map=color_map
)


fig.update_traces(marker=dict(cornerradius=3))
fig.show()

::: column-screen

In [ ]:
m = leafmap.Map(
    center=(center[1], center[0]),
    zoom=10,
    draw_control=False,
    measure_control=False,
    fullscreen_control=False,
    attribution_control=False,
    search_control=False,
    layers_control=False,
    scale_control=False,
    toolbar_control=False,
    basemap="Esri.WorldImagery"
)
m.add_gdf(center_point_gdf, layer_name="Buffer", zoom_to_layer=True)
m.add_geojson(geometries, layer_name="Your Parcels")

m.add_geojson(gdbf_df[['iucnRedListCategory', "kingdom", "class", "species", "geometry"]].__geo_interface__, layer_name="Endangered Species", info_mode="on_hover", hover_style={"fillColor": "red", "fillOpacity": 0.5})
m

:::

### Protected Areas
> Protected areas are regions set aside to preserve the environment and biodiversity. They can be national parks, wildlife sanctuaries, or marine reserves. 


In [ ]:
from IPython.display import Markdown
import requests
from bs4 import BeautifulSoup
ramsar_sites = gpd.read_parquet("./data/features_centroid_publishedPoint.parquet").to_crs("EPSG:3857")
center_point_df = gpd.GeoDataFrame(geometry=[Point(center)], crs="EPSG:4326").to_crs("EPSG:3857")
# get 5 nearest ramsar sites and their distance
nearest_ramsar_sites = gpd.sjoin_nearest(ramsar_sites, center_point_df, how="inner", max_distance=None, distance_col="distance").nsmallest(5, 'distance')

def get_description(idx):
  url = f'https://rsis.ramsar.org/ris/{idx}'
  response = requests.get(url)
  if response.status_code != 200:
    return None
  soup = BeautifulSoup(response.content, 'html.parser')

  summary = soup.find('div', {'class': 'field-name-asummary'}).get_text(strip=True)
  return summary


nearest_ramsar_sites
for index, row in nearest_ramsar_sites.iterrows():
  description = get_description(row["ramsarid"])
  display(Markdown(f"#### {row['officialna']} - ({round(row['distance'] / 1000)} km)"))
  display(Markdown(description))

::: column-screen

In [ ]:
m = leafmap.Map(
    center=(center[1], center[0]),
    zoom=9,
    draw_control=False,
    measure_control=False,
    fullscreen_control=False,
    attribution_control=False,
    search_control=False,
    layers_control=False,
    scale_control=False,
    toolbar_control=False,
    basemap="Esri.WorldGrayCanvas"
)
ramsar_url = "https://rsis.ramsar.org/geoserver/wms?"
m.add_wms_layer(
    url=ramsar_url, layers="ramsar_sdi:features", name="NAIP Imagery", format="image/png", shown=True, srs="EPSG:4326", zoom_to_layer=False
)
m.add_wms_layer(
    url=ramsar_url, layers="ramsar_sdi:features_centroid", name="NAIP Imagery", format="image/png", shown=True, srs="EPSG:4326", zoom_to_layer=False
)
m.add_geojson(geometries, layer_name="Your Parcels", zoom_to_layer=False)


m.add_tile_layer(
  url="https://data-gis.unep-wcmc.org/server/rest/services/ProtectedSites/The_World_Database_of_Protected_Areas/MapServer/tile/{z}/{y}/{x}",
  attribution="WDPA",
  name="WDPA"
)
m

:::


In [ ]:
import rasterstats

url = "http://54.229.242.119/GSOCseqv1.1/GSOCseq_T0_Map030.tif"
stats = rasterstats.zonal_stats(geometries, url, stats=["mean"], all_touched=True)
stats

In [ ]:
m = leafmap.Map(
    center=(center[1], center[0]),
    zoom=9,
    draw_control=False,
    measure_control=False,
    fullscreen_control=False,
    attribution_control=False,
    search_control=False,
    layers_control=False,
    scale_control=False,
    toolbar_control=False,
    basemap="Esri.WorldGrayCanvas"
)


m.add_wms_layer(
    url=url, layers="RSR", name="NAIP Imagery", format="image/png", shown=True, srs="EPSG:3857", zoom_to_layer=False
)
m

## Soical Indices
### Land and Environmental Related Killings
> Global Witness defines land and environmental defenders as individuals who take peaceful action against the exploitation of natural resources and the environment, often at significant personal risk. Their methodology includes reviewing credible online sources and collaborating with regional partners, though they acknowledge their data likely underestimates the true extent of these killings due to unreported cases.


In [ ]:
import leafmap
import geopandas as gpd

countries = gpd.read_file("./data/world_countries_geojson.gpkg")
countries["un_a3"] = countries["un_a3"].astype(int)
countries["centroid"] = countries.centroid
global_witness = pd.read_csv("./data/global_witness_led_10-10-24.csv")
global_witness = gpd.GeoDataFrame(global_witness.merge(countries[["un_a3", "centroid"]], left_on="country_numeric", right_on="un_a3"), crs="EPSG:4326", geometry="centroid")
global_witness["latitude"] = global_witness["centroid"].y
global_witness["longitude"] = global_witness["centroid"].x

m = leafmap.Map(
    center=(center[1], center[0]),
    zoom=2,
    draw_control=False,
    measure_control=False,
    fullscreen_control=False,
    attribution_control=False,
    search_control=False,
    layers_control=False,
    scale_control=False,
    toolbar_control=False,
    basemap="Esri.WorldGrayCanvas"
)
# m.add_points_from_xy(
#   global_witness[["country", "name", "gender", "industry_driver", "local_region", "longitude", "latitude"]],
#   x="longitude",
#   y="latitude",
#   icon_names=["heart"],
# )
m.add_point_layer(
  global_witness,
  popup=["country", "name", "gender", "industry_driver", "local_region"]
)
m